In [1]:
import joblib
from sklearn_crfsuite import CRF

class CustomCRF(CRF):
    def feature_dict(self, x, y=None, state_features=None):
        features = super().feature_dict(x, y, state_features)
        return features
crf = joblib.load('crf_model_with_induction.joblib')


In [2]:
def word2features(sent, i,pos_tags_,entity_patterns,token_list,chunks):
    word = sent[i]
    
    if pos_tags_ != None:
        postag = pos_tags_[i]
    if pos_tags_ != None:
        features = {
            'bias': 1.0,
            'word.lower()': word.lower(),
            'postag': postag,
            'word[-3:]': word[-3:],
            'word[-2:]': word[-2:],
            'word.isupper()': word.isupper(),
            'word.istitle()': word.istitle(),
            'word.isdigit()': word.isdigit(),
            'word.first_letter': word[0],
            'word.last_letter': word[-1],
            'has_ingsuffix': word.endswith('ing'),
            'prev_word': sent[i-1] if i > 0 else '<START>',
            'next_word': sent[i+1] if i < len(sent)-1 else '<END>',
            'prev_word_is_upper': sent[i-1].isupper() if i > 0 else False,
            'next_word_is_upper': sent[i+1].isupper() if i < len(sent)-1 else False,
            'pos_prefix': postag[:2],
            'pos_suffix': postag[-2:],
            'pos_category': postag[0],
            'word.is_all_uppercase': word.isupper(),
            'word.is_all_lowercase': word.islower(),
            'word.is_mixed_case': not word.isupper() and not word.islower(),
            'chunk':chunks[i],
            'prev_chunk': chunks[i-1] if i > 0 else '<START>',
            'next_chunk': chunks[i+1] if i < len(sent)-1 else '<END>',
            # 'induced_pattern_1': check_for_induced_pattern_1(word, sent),
            # 'induced_pattern_2': check_fors_induced_pattern_2(word, sent),
            # add additional induced patterns as necessary
        }
    else:
        features = {
            'bias': 1.0,
            'word.lower()': word.lower(),
            'word[-3:]': word[-3:],
            'word[-2:]': word[-2:],
            'word.isupper()': word.isupper(),
            'word.istitle()': word.istitle(),
            'word.isdigit()': word.isdigit(),
            'word.first_letter': word[0],
            'word.last_letter': word[-1],
            'has_ingsuffix': word.endswith('ing'),
            'prev_word': sent[i-1] if i > 0 else '<START>',
            'next_word': sent[i+1] if i < len(sent)-1 else '<END>',
            'prev_word_is_upper': sent[i-1].isupper() if i > 0 else False,
            'next_word_is_upper': sent[i+1].isupper() if i < len(sent)-1 else False,
            'word.is_all_uppercase': word.isupper(),
            'word.is_all_lowercase': word.islower(),
            'word.is_mixed_case': not word.isupper() and not word.islower(),
            # 'induced_pattern_1': check_for_induced_pattern_1(word, sent),
            # 'induced_pattern_2': check_fors_induced_pattern_2(word, sent),
            # add additional induced patterns as necessary
        }
    if entity_patterns != None:
        token = token_list[i]
        passed_patterns = []
        for entity_in,patterns in entity_patterns.items():
            if entity_in in ' '.join(sent):
                if word in entity_in:
                    if token != 'O':
                        # print(' '.join(sent))
                        #print(entity_in)
                        # print(patterns)
                        passed_patterns.append(patterns)

        if len(passed_patterns)>0:
            features['extracted entity'] = True
            for i in range(len(passed_patterns)):
                for j in range(len(passed_patterns[i])):
                    features[f'pattern_{i*len(passed_patterns)+j}']=passed_patterns[i][j]
        else:
            features['extracted entity'] = False
             
        #print(word,passed_patterns)
    return features

In [12]:
import nltk
nltk.download('punkt')

#tokens = nltk.word_tokenize("Lorenzo goes to the University of Illinois at Urbana-Champaign")
tokens = nltk.word_tokenize("He went to the Apple store.")

features = [word2features(tokens, index,pos_tags_=None,entity_patterns = None,token_list=None,chunks=None) for index in range(len(tokens))]
pred_labels = crf.predict([features])[0]
print(pred_labels)
print(tokens)
for i in range(len(pred_labels)):
    print(pred_labels[i], tokens[i])

['O', 'O', 'O', 'O', 'B-ORG', 'O', 'O']
['He', 'went', 'to', 'the', 'Apple', 'store', '.']
O He
O went
O to
O the
B-ORG Apple
O store
O .


[nltk_data] Downloading package punkt to /Users/lorenzo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# With Induction
              precision    recall  f1-score   support

       B-LOC       0.88      0.87      0.88      1266
      B-MISC       0.84      0.83      0.83       563
       B-ORG       0.86      0.80      0.83      1229
       B-PER       0.82      0.92      0.87      1025
       I-LOC       0.85      0.72      0.78       220
      I-MISC       0.73      0.67      0.70       162
       I-ORG       0.82      0.75      0.78       515
       I-PER       0.85      0.99      0.92       720
           O       1.00      0.99      1.00      3411

   micro avg       0.90      0.90      0.90      9111

   macro avg       0.85      0.84      0.84      9111

weighted avg       0.90      0.90      0.90      9111

 samples avg       0.92      0.91      0.91      9111

In [11]:
crf = joblib.load('crf_model_without_induction.joblib')

In [12]:
import nltk
nltk.download('punkt')

tokens = nltk.word_tokenize("Lorenzo goes to the University of Illinois at Urbana-Champaign")

features = [word2features(tokens, index,pos_tags_=None,entity_patterns = None,token_list=None,chunks=None) for index in range(len(tokens))]
pred_labels = crf.predict([features])[0]
print(pred_labels)
print(tokens)
for i in range(len(pred_labels)):
    print(pred_labels[i], tokens[i])

['O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'B-LOC']
['Lorenzo', 'goes', 'to', 'the', 'University', 'of', 'Illinois', 'at', 'Urbana-Champaign']
O Lorenzo
O goes
O to
O the
B-ORG University
I-ORG of
I-ORG Illinois
O at
B-LOC Urbana-Champaign


[nltk_data] Downloading package punkt to /Users/lorenzo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Without Induction
              precision    recall  f1-score   support

       B-LOC       0.88      0.87      0.88      1266
      B-MISC       0.82      0.83      0.82       563
       B-ORG       0.86      0.80      0.83      1229
       B-PER       0.83      0.92      0.87      1025
       I-LOC       0.80      0.78      0.79       220
      I-MISC       0.66      0.70      0.68       162
       I-ORG       0.79      0.77      0.78       515
       I-PER       0.86      0.98      0.92       720
           O       1.00      0.99      1.00      3411

   micro avg       0.90      0.91      0.90      9111

   macro avg       0.83      0.85      0.84      9111
   
weighted avg       0.90      0.91      0.90      9111

 samples avg       0.91      0.91      0.90      9111

https://paperswithcode.com/sota/named-entity-recognition-ner-on-conll-2003?p=automated-concatenation-of-embeddings-for-1